In [17]:
import pandas as pd
import requests

In [18]:
df = pd.read_csv('..\\..\\camada_bronze\\Crude Oil WTI Futures Historical Data.csv')

df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,12/31/2024,71.25,70.69,71.52,70.43,63.63K,1.02%
1,12/30/2024,70.53,70.00,71.05,69.75,93.10K,0.50%
2,12/27/2024,70.18,69.27,70.32,69.06,79.13K,1.37%
3,12/26/2024,69.23,69.72,70.25,68.94,69.64K,-0.59%
4,12/24/2024,69.64,69.04,69.94,68.95,40.39K,1.21%


In [20]:
# Converter a coluna 'Date' para o formato correto
df['Date'] = pd.to_datetime(df['Date'])

print("Dados após a modificação:")
df.head()


Dados após a modificação:


,Date,Price,Open,High,Low,Vol.,Change %
0,2024-12-31,71.25,70.69,71.52,70.43,63.63K,1.02%
1,2024-12-30,70.53,70.00,71.05,69.75,93.10K,0.50%
2,2024-12-27,70.18,69.27,70.32,69.06,79.13K,1.37%
3,2024-12-26,69.23,69.72,70.25,68.94,69.64K,-0.59%
4,2024-12-24,69.64,69.04,69.94,68.95,40.39K,1.21%


In [21]:
df['Vol.'] = df['Vol.'].str[:-1]
df = df.rename(columns={"Vol.":"Volume"})
df.head()

,Date,Price,Open,High,Low,Volume,Change %
0,2024-12-31,71.25,70.69,71.52,70.43,63.63,1.02%
1,2024-12-30,70.53,70.00,71.05,69.75,93.10,0.50%
2,2024-12-27,70.18,69.27,70.32,69.06,79.13,1.37%
3,2024-12-26,69.23,69.72,70.25,68.94,69.64,-0.59%
4,2024-12-24,69.64,69.04,69.94,68.95,40.39,1.21%


In [22]:
df.isna().sum()

Date          0
Price         0
Open          0
High          0
Low           0
Volume      130
Change %      0
dtype: int64

In [23]:
df["Volume"] = df["Volume"].apply(lambda x: pd.to_numeric(x))

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3907 entries, 0 to 3906
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      3907 non-null   datetime64[ns]
 1   Price     3907 non-null   float64       
 2   Open      3907 non-null   float64       
 3   High      3907 non-null   float64       
 4   Low       3907 non-null   float64       
 5   Volume    3777 non-null   float64       
 6   Change %  3907 non-null   object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 213.8+ KB


In [25]:
list_index = [i for i in df[pd.isna(df["Volume"])].index]

In [26]:
for i in list_index:
    df.loc[i, "Volume"] = df[~pd.isna(df["Volume"])].iloc[i-2:i+2]["Volume"].mean()

In [27]:
df.iloc[84]

Date        2024-09-02 00:00:00
Price                     73.78
Open                       73.0
High                      74.39
Low                       72.89
Volume                 225.3775
Change %                  1.10%
Name: 84, dtype: object

In [32]:
def get_cotacao_dolar(data):
    # Garantir que a data seja convertida para datetime, caso não seja
    if isinstance(data, str):
        data = pd.to_datetime(data)
    
    # Converter o Timestamp para string no formato YYYY-MM-DD
    data_formatada = data.strftime('%Y-%m-%d')  # Garantir que a data está no formato correto
    
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.1/dados?dataInicial={data_formatada}&dataFinal={data_formatada}&formato=json"
    response = requests.get(url)
    
    # Imprimir a resposta bruta para verificar o que a API está retornando
    print(f"URL: {url}")
    print(f"Status Code: {response.status_code}")
    print(f"Resposta: {response.text}")  # Exibe o conteúdo da resposta
    
    if response.status_code == 200:
        try:
            data_dolar = response.json()
            # Verificar se a resposta está vazia
            if data_dolar:
                return float(data_dolar[0]['valor'].replace(',', '.'))
            else:
                print(f"Nenhum dado encontrado para a data {data_formatada}")
                return None  # Caso não tenha dados para aquela data
        except Exception as e:
            print(f"Erro ao tentar decodificar JSON para a data {data_formatada}: {e}")
    else:
        print(f"Erro na requisição para a data {data_formatada}, código de status: {response.status_code}")
    return None

# Carregar o arquivo CSV
df = pd.read_csv('..\\..\\camada_bronze\\Crude Oil WTI Futures Historical Data.csv')

# Garantir que a coluna 'Date' seja convertida para datetime
df['Date'] = pd.to_datetime(df['Date'])

# Adicionar uma nova coluna 'PriceR$' para o preço em reais, ignorando datas com erro
df['PriceR$'] = df['Date'].apply(lambda x: get_cotacao_dolar(x)).multiply(df['Price'])

# Renomear as colunas conforme solicitado
df.rename(columns={'Price': 'Price$', 'PriceR$': 'Price'}, inplace=True)

display(df)

URL: https://api.bcb.gov.br/dados/serie/bcdata.sgs.1/dados?dataInicial=2024-12-31&dataFinal=2024-12-31&formato=json
Status Code: 400
Resposta: {"erro":{"statusCode":400,"detail":"br.gov.bcb.pec.sgs.comum.excecoes.SGSNegocioException: Invalid initial date"}}
Erro na requisição para a data 2024-12-31, código de status: 400
URL: https://api.bcb.gov.br/dados/serie/bcdata.sgs.1/dados?dataInicial=2024-12-30&dataFinal=2024-12-30&formato=json
Status Code: 400
Resposta: {"erro":{"statusCode":400,"detail":"br.gov.bcb.pec.sgs.comum.excecoes.SGSNegocioException: Invalid initial date"}}
Erro na requisição para a data 2024-12-30, código de status: 400
URL: https://api.bcb.gov.br/dados/serie/bcdata.sgs.1/dados?dataInicial=2024-12-27&dataFinal=2024-12-27&formato=json
Status Code: 400
Resposta: {"erro":{"statusCode":400,"detail":"br.gov.bcb.pec.sgs.comum.excecoes.SGSNegocioException: Invalid initial date"}}
Erro na requisição para a data 2024-12-27, código de status: 400
URL: https://api.bcb.gov.br/dad

KeyboardInterrupt: 

In [ ]:
# Salvar o DataFrame modificado
modified_file_path = '..\\..\\camada_prata\\Crude_Oil_clean.csv'
df.to_csv(modified_file_path, index=False)

Dados após a modificação:


In [16]:
display(df)

,Date,Price,Open,High,Low,Volume,Change %
0,2024-12-31,71.25,70.69,71.52,70.43,63.63,1.02%
1,2024-12-30,70.53,70.00,71.05,69.75,93.10,0.50%
2,2024-12-27,70.18,69.27,70.32,69.06,79.13,1.37%
3,2024-12-26,69.23,69.72,70.25,68.94,69.64,-0.59%
4,2024-12-24,69.64,69.04,69.94,68.95,40.39,1.21%
...,...,...,...,...,...,...,...
3902,2010-01-08,82.75,82.65,83.47,81.80,310.38,0.11%
3903,2010-01-07,82.66,83.20,83.36,82.26,246.63,-0.63%
3904,2010-01-06,83.18,81.43,83.52,80.85,370.06,1.72%
3905,2010-01-05,81.77,81.63,82.00,80.95,258.89,0.32%


In [14]:
df.isna().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Volume      0
Change %    0
dtype: int64